In [1]:
import sys
!{sys.executable} -m pip install gdown h5py

In [2]:
from utils import load_it_data, visualize_img
import matplotlib.pyplot as plt
import numpy as np
import gdown
url = "https://drive.google.com/file/d/1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS/view?usp=share_link"
output = "IT_data.h5"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS
From (redirected): https://drive.google.com/uc?id=1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS&confirm=t&uuid=c76db502-df83-40e0-9fc7-b9b7ce0e5c17
To: /home/jupyter/IT_data.h5
100%|██████████| 384M/384M [00:13<00:00, 29.4MB/s] 


'IT_data.h5'

In [3]:
from utils import load_it_data, visualize_img
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import torch
from torch import Tensor
from torchvision.models import ResNet, resnet50, ResNet50_Weights
import pickle
import gc
from tqdm import tqdm
import pandas as pd

In [4]:
path_to_data = '' ## Insert the folder where the data is, if you download in the same folder as this notebook then leave it blank

stimulus_train, _, _, _, _, _, spikes_train, _ = load_it_data(path_to_data)
layers = ["conv1", "layer1", "layer2", "layer3", "layer4", "avgpool"]

In [5]:
def apply_PCA(activations, n_components = 1000) :
    """apply PCA on the activations of a layer

    Args:
        layer_file (string): name of the layer where the activations data are extracted
        n_components (int): number of components we want to keep

    Returns:
        activations: computed PC from the activations
    """
    pca = PCA(n_components=n_components)
    pca.fit(activations)
    activations = pca.transform(activations)
    return activations

In [14]:
def extract_activation(self, stimuli: Tensor) :
    """extract the activations of the model for the given stimuli and layer

    Args:
        model (model): model we want to extract the activations from
        stimuli (ndarray): input data of the processed image's pixels

    Returns:
        dict: dictionary containing the activations for each layer of the model
    """    
    activations = []
    for x in tqdm(stimuli) : 
        x = self.conv1(x.unsqueeze(0))
        activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
    with open('conv1.pkl','wb') as f:
        pickle.dump(apply_PCA(activations), f)
    
    input_ = activations
    activations = []
    for x in tqdm(input_) :
        x = self.bn1(x.unsqueeze(0))
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
    with open('layer1.pkl','wb') as f:
        pickle.dump(apply_PCA(activations), f)
    
    activations = []
    for x in tqdm(stimuli) :
        x = self.layer2(x.unsqueeze(0))
        activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
    with open('layer2.pkl','wb') as f:
        pickle.dump(apply_PCA(activations), f)    

    input_ = activations
    activations = []
    for x in tqdm(input_) :
        x = self.layer3(x.unsqueeze(0))
        activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
    with open('layer3.pkl','wb') as f:
        pickle.dump(apply_PCA(activations), f)
    
    input_ = activations
    activations = []
    for x in tqdm(input_) :
        x = self.layer4(x.unsqueeze(0))
        activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
    with open('layer4.pkl','wb') as f:
        pickle.dump(apply_PCA(activations), f)
    
    input_ = activations
    activations = []
    for x in tqdm(input_) :
        x = self.avgpool(x.unsqueeze(0))
        activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
    with open('avgpool.pkl','wb') as f:
        pickle.dump(apply_PCA(activations), f)
        
ResNet.extract_activation = extract_activation

In [15]:
# load the pre-trained ResNet50 model
stimulus_train = torch.tensor(stimulus_train)
model = resnet50(weights=ResNet50_Weights.DEFAULT) # include_top = False?
model.eval()
print()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [16]:
#Preprocess the stimuli (already done)
"""preprocess = weights.transforms()
img_transformed = preprocess(stimuli)
"""

'preprocess = weights.transforms()\nimg_transformed = preprocess(stimuli)\n'

In [17]:
# extract the activations of the layers and apply PCA on each layer to store the first 1000PCs
model.extract_activation(stimulus_train)

  0%|          | 0/2592 [00:00<?, ?it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'